# News Pre processing

In this notebook we are going to pre process the news dataset, categorizing the properties and vectorizing the text.

## Importing the libraries and Fasttext model

First we are going to import all the libraries needed and the model to be uses to embbed the texts.

In [1]:
import json
import numpy as np
import pandas as pd
import unidecode
from pandas.io.json import json_normalize
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models import KeyedVectors

word_vec_model = KeyedVectors.load_word2vec_format('../datasets/word-vectors/wiki.pt/wiki.pt.vec')

word_vec_model

Slow version of gensim.models.doc2vec is being used


## Creating the news dataframe
The first step is import the news dataset and creating a Pandas Dataframe

In [16]:
news_data_file = open('../datasets/raw-data/folha_de_sao_paulo_news.json', 'r')
news_json = json.load(news_data_file)

news_df = json_normalize(news_json['news'])

news_df.head(5)

,authors,categories,date_published,link,locations,news_body,sub_title,title
0,[thiago resende],"[mercado, indústria_4.0, tec, folhainvest, mer...",2019-06-27 21:01:00,https://www1.folha.uol.com.br/mercado/2019/06/...,[brasília],"uma conjunção de fatores, como de categorias, ...","perto da reta final em comissão na câmara, pro...","desarticulação, pressão de partidos e lobby de..."
1,"[camila mattoso, ranier bragon]","[poder, governo_bolsonaro, lava_jato, legislat...",2019-06-27 07:54:00,https://www1.folha.uol.com.br/poder/2019/06/pf...,[brasília],a polícia federal prendeu na manhã desta quint...,ação deflagrada mira candidaturas laranjas pat...,pf prende assessores de ministro do turismo em...
2,[NO AUTHOR],"[mercado, indústria_4.0, tec, folhainvest, mer...",2019-06-27 10:41:00,https://www1.folha.uol.com.br/mercado/2019/06/...,"[paris, reuters]","o presidente francês, emmanuel macron, afirmou...",frança está preocupada com impacto sobre sua i...,macron diz que não terá acordo com mercosul se...
3,[júlia moura],"[mercado, indústria_4.0, tec, folhainvest, mer...",2019-06-27 17:33:00,https://www1.folha.uol.com.br/mercado/2019/06/...,[são_paulo],o e a indecisão sobre a inclusão de estados e ...,"ibovespa chegou a recuar 1,26% nesta quinta, m...",maia contém apreensão com atraso na previdênci...
4,[daniel carvalho],"[mercado, indústria_4.0, tec, folhainvest, mer...",2019-06-27 16:58:00,https://www1.folha.uol.com.br/mercado/2019/06/...,[brasília],"o ministro da economia, paulo guedes, disse ne...",presidente do senado disse que haverá conversa...,guedes defende estados na previdência para que...


# Categorizing the properties

Now we are going to tranform the categoric values in columns in the dataframe.

## First step - Splitting the list values in multiple rows

The first step is to split the list values in the categories columns in multiple rows, so we can use Pandas to convert it into columns. Additionally, we are removing the author column for now as it would multiple the number of columns a lot.

In [17]:
news_df.reset_index(inplace=True, drop=True)
rows = []
_ = news_df.apply(
    lambda row: [
        rows.append(
            [ 
                unidecode.unidecode(category).lower(), 
                row['date_published'], 
                unidecode.unidecode(location).lower(), 
                row['sub_title'], 
                row['title']
            ]
        ) 
        for category in row.categories
        for location in row.locations
    ], axis=1)
news_df = pd.DataFrame(rows, columns=['category', 'date_published', 'location', 'sub_title', 'title'])

news_df.head(10)

,category,date_published,location,sub_title,title
0,mercado,2019-06-27 21:01:00,brasilia,"perto da reta final em comissão na câmara, pro...","desarticulação, pressão de partidos e lobby de..."
1,industria_4.0,2019-06-27 21:01:00,brasilia,"perto da reta final em comissão na câmara, pro...","desarticulação, pressão de partidos e lobby de..."
2,tec,2019-06-27 21:01:00,brasilia,"perto da reta final em comissão na câmara, pro...","desarticulação, pressão de partidos e lobby de..."
3,folhainvest,2019-06-27 21:01:00,brasilia,"perto da reta final em comissão na câmara, pro...","desarticulação, pressão de partidos e lobby de..."
4,mercado_financeiro,2019-06-27 21:01:00,brasilia,"perto da reta final em comissão na câmara, pro...","desarticulação, pressão de partidos e lobby de..."
5,previdencia,2019-06-27 21:01:00,brasilia,"perto da reta final em comissão na câmara, pro...","desarticulação, pressão de partidos e lobby de..."
6,arena_do_marketing,2019-06-27 21:01:00,brasilia,"perto da reta final em comissão na câmara, pro...","desarticulação, pressão de partidos e lobby de..."
7,mpme,2019-06-27 21:01:00,brasilia,"perto da reta final em comissão na câmara, pro...","desarticulação, pressão de partidos e lobby de..."
8,mercado,2019-06-27 21:01:00,brasilia,"perto da reta final em comissão na câmara, pro...","desarticulação, pressão de partidos e lobby de..."
9,industria_4.0,2019-06-27 21:01:00,brasilia,"perto da reta final em comissão na câmara, pro...","desarticulação, pressão de partidos e lobby de..."


## Second step - Converting the categories into columns

Then we are using the get_dummies method from pandas to convert each categoric value in the 'category' and 'location' columns into a new column with a boolean value(0 if it does not belong to the category and 1 if it does).

In [18]:
categorized = pd.get_dummies(news_df, prefix=['category', 'location'], columns=['category', 'location']).drop_duplicates(keep='first') 

categorized.head(5)

,date_published,sub_title,title,category_agencia_lupa,category_arena_do_marketing,category_brexit,category_coreia_do_norte,category_entrevista_da_2a,category_folhainvest,category_governo_bolsonaro,...,location_teresina,location_terra_indigena_raposa_serra_do_sol,location_the_new_york_times,location_uberaba,location_uol,location_vitoria,location_washington,location_washington_post,location_xangai,location_xapuri
0,2019-06-27 21:01:00,"perto da reta final em comissão na câmara, pro...","desarticulação, pressão de partidos e lobby de...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2019-06-27 21:01:00,"perto da reta final em comissão na câmara, pro...","desarticulação, pressão de partidos e lobby de...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2019-06-27 21:01:00,"perto da reta final em comissão na câmara, pro...","desarticulação, pressão de partidos e lobby de...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2019-06-27 21:01:00,"perto da reta final em comissão na câmara, pro...","desarticulação, pressão de partidos e lobby de...",0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,2019-06-27 21:01:00,"perto da reta final em comissão na câmara, pro...","desarticulação, pressão de partidos e lobby de...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Grouping the news categories

Now we group all the news categories

In [19]:
categorized = categorized.groupby(['date_published', 'title', 'sub_title'], as_index=False).agg('max')

categorized.head(5)

,date_published,title,sub_title,category_agencia_lupa,category_arena_do_marketing,category_brexit,category_coreia_do_norte,category_entrevista_da_2a,category_folhainvest,category_governo_bolsonaro,...,location_teresina,location_terra_indigena_raposa_serra_do_sol,location_the_new_york_times,location_uberaba,location_uol,location_vitoria,location_washington,location_washington_post,location_xangai,location_xapuri
0,2018-07-01 02:00:00,meirelles encara eleição como última chance e ...,"segundo aliados, ex-ministro investe r$ 250 mi...",1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,2018-07-03 02:00:00,"bank of america projeta dólar em r$ 5,50 depoi...",cenário crítico ocorreria caso o novo presiden...,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,2018-07-03 02:00:00,candidato do prtb em sp defende máquina menor,o advogado rodrigo tavares promete cortar pela...,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,2018-07-03 02:00:00,distanciamento de alckmin em são paulo esfria ...,entorno do governador vê com ceticismo chance ...,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,2018-07-03 19:05:00,indústria apresenta proposta de reforma tribut...,cni quer isenção na compra de máquinas e ampli...,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


## Calculating the centroid of the related words

In [20]:
def embed_sentences(sentences):
    embedded_sentences = []
    for sentence in sentences:
        vectors = []
        for token in sentence:
            if token in word_vec_model.vocab:
                vectors.append(word_vec_model[token])
                
        embedded_sentences.append(np.sum(vectors, axis=0) / len(vectors[0]))
            
    return embedded_sentences

In [21]:
related_terms = [
    ['inflação'],
    ['ipca'],
    ['exportação'] , 
    ['importação'], 
    ['déficit'], 
    ['superávit'], 
    ['gastos'],  
    ['taxa', 'de', 'juros'], 
    ['selic'],
    ['banco', 'central'],
    ['dívida', 'pública'],
    ['recessão'], 
    ['dólar'],
    ['investidores'],
    ['investimento'],
    ['previdência'],
    ['reforma', 'da', 'previdência'],
    ['investigação'],
    ['pib'],
    ['crédito'],
    ['imigração'],
    ['dívida'],
    ['subsídios'],
    ['gastos']
]

related_terms_embbedings = embed_sentences(related_terms)

centroid = np.sum(related_terms_embbedings, axis=0) / len(related_terms_embbedings[0])

centroid

array([ 2.51559886e-05,  4.09876229e-05, -7.62913842e-05, -9.76656302e-05,
        9.49627865e-05,  8.68028146e-05,  1.06093612e-05, -2.36718006e-05,
        4.48804640e-05, -4.71247113e-05,  1.64947432e-05, -2.72350353e-05,
        6.41097722e-05,  1.55752023e-05, -2.97714141e-05, -7.05136990e-05,
        5.47878008e-05, -1.29627544e-04,  2.80864224e-05, -9.23271364e-05,
        1.68304778e-05, -9.11800744e-05,  5.12964907e-05,  2.71049976e-05,
       -1.75273567e-06,  2.32263218e-07,  1.01253017e-05, -1.29201115e-04,
        8.46142430e-05, -3.71427523e-05,  5.63696012e-05, -8.25289826e-05,
       -6.68990688e-05,  6.58525387e-05, -1.18135285e-04, -3.30012772e-05,
       -4.86091376e-05, -1.54849567e-05, -1.38930845e-04, -6.96914867e-05,
        4.00645404e-05,  4.94683336e-05,  5.78575091e-05,  1.68891129e-05,
        1.78614646e-05, -1.05534236e-05,  5.71582277e-05,  5.75975573e-05,
        3.93478376e-06, -4.68473299e-05, -2.23424763e-06, -1.40745678e-05,
        1.33010108e-05, -

## Embedding vord vectors of the new's title and subtitle

In [25]:
from scipy.spatial.distance import cdist

def embed_sentence(sentence):
    if not sentence:
        return 0
    
    sentence_tokens = [t for t in sent_tokenize(sentence, language='portuguese')]
    vectors = []
    for sentence in sentence_tokens:
        for token in sentence:
            if token in word_vec_model.vocab:
                vectors.append([word_vec_model[token]])
    return np.array(vectors)

def mean_embed_sentence(sentence):
    if not sentence:
        return 0
    embedded_sentence = embed_sentence(sentence)

    sentence_centroid = np.sum(embedded_sentence, axis=0) / len(embedded_sentence[0])
    return sentence_centroid

def calculate_centroids_distance(sentence):
    if not sentence:
        return 0
    
    vectors = embed_sentence(sentence)
    sentence_centroid = np.sum(vectors, axis=0) / len(vectors[0])
    centroids = [sentence_centroid[0], centroid]
    centroids_distance = cdist(centroids, centroids, 'cosine')
    centroids_distance_mod = np.sqrt(np.sum(np.multiply(centroids_distance, centroids_distance)))
    
    return 1 / centroids_distance_mod

categorized['news_title_embedded'] = categorized.title.apply(mean_embed_sentence)
categorized['news_title_inverse_centroid'] = categorized.title.apply(calculate_centroids_distance)
categorized['news_subtitle_embedded'] = categorized.sub_title.apply(mean_embed_sentence)
categorized['news_subtitle_inverse_centroid'] = categorized.sub_title.apply(calculate_centroids_distance)

categorized.head(5)

,date_published,title,sub_title,category_agencia_lupa,category_arena_do_marketing,category_brexit,category_coreia_do_norte,category_entrevista_da_2a,category_folhainvest,category_governo_bolsonaro,...,location_uol,location_vitoria,location_washington,location_washington_post,location_xangai,location_xapuri,news_title_embedded,news_title_inverse_centroid,news_subtitle_embedded,news_subtitle_inverse_centroid
0,2018-07-01 02:00:00,meirelles encara eleição como última chance e ...,"segundo aliados, ex-ministro investe r$ 250 mi...",1,0,0,0,1,0,1,...,0,0,0,0,0,0,"[[1.1757944, 4.230781, -18.693535, -7.810856, ...",1.168745,"[[2.2614148, 2.6860142, -24.55114, -10.085097,...",1.132379
1,2018-07-03 02:00:00,"bank of america projeta dólar em r$ 5,50 depoi...",cenário crítico ocorreria caso o novo presiden...,0,1,0,0,0,1,0,...,0,0,0,0,0,0,"[[0.84330237, 4.512057, -16.2687, -3.8113544, ...",1.199275,"[[3.2768018, 5.9720984, -20.550531, -7.522219,...",1.134610
2,2018-07-03 02:00:00,candidato do prtb em sp defende máquina menor,o advogado rodrigo tavares promete cortar pela...,1,0,0,0,1,0,1,...,0,0,0,0,0,0,"[[-0.115924545, 0.93324816, -14.384352, -4.821...",1.131943,"[[1.4467132, 2.6573582, -23.472956, -9.3213825...",1.143894
3,2018-07-03 02:00:00,distanciamento de alckmin em são paulo esfria ...,entorno do governador vê com ceticismo chance ...,1,0,0,0,1,0,1,...,0,0,0,0,0,0,"[[3.3298578, 4.3628135, -18.711407, -6.250186,...",1.142421,"[[0.24959418, 4.880267, -25.270489, -9.932234,...",1.130034
4,2018-07-03 19:05:00,indústria apresenta proposta de reforma tribut...,cni quer isenção na compra de máquinas e ampli...,0,1,0,0,0,1,0,...,0,0,0,0,0,0,"[[-2.3236966, 3.9482424, -23.047253, -7.643917...",1.098769,"[[1.9849598, 4.306823, -21.564844, -6.9355807,...",1.141967


## Saving the pre-processed dataset in a csv file

In [26]:
categorized.to_csv('../datasets/pre-processed/news.csv', index= False, encoding='utf-8')